In [13]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [14]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-11-14 20:04:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  85.1MB/s    in 0.3s    

2024-11-14 20:04:59 (85.1 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [19]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [20]:
# Filter out users and books based on rating counts
filtered_users = df_ratings['user'].value_counts()
filtered_users = filtered_users[filtered_users >= 200].index
filtered_books = df_ratings['isbn'].value_counts()
filtered_books = filtered_books[filtered_books >= 100].index

# Apply the filter
filtered_ratings = df_ratings[(df_ratings['user'].isin(filtered_users)) & (df_ratings['isbn'].isin(filtered_books))]

# Create a pivot table for the user-book matrix
book_user_matrix = filtered_ratings.pivot(index='isbn', columns='user', values='rating').fillna(0)

# Convert the pivot table to a sparse matrix format
book_user_sparse = csr_matrix(book_user_matrix.values)

# Train NearestNeighbors model using cosine similarity
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_user_sparse)

# Create mappings between ISBN and book title
isbn_to_title = dict(zip(df_books['isbn'], df_books['title']))
title_to_isbn = dict(zip(df_books['title'], df_books['isbn']))

In [21]:
def get_recommends(book=""):
    # Check if the book is in the dataset
    if book not in title_to_isbn:
        return [book, []]

    # Get the ISBN of the requested book
    book_isbn = title_to_isbn[book]

    # Get the index of the book
    book_idx = book_user_matrix.index.get_loc(book_isbn)

    # Find the nearest neighbors for the book
    distances, indices = model_knn.kneighbors(book_user_sparse[book_idx], n_neighbors=6)

    # Prepare the recommendations list
    recommended_books = [book]
    similar_books = []

    for i in range(1, len(distances.flatten())):
        similar_idx = indices.flatten()[i]
        similar_isbn = book_user_matrix.index[similar_idx]
        similar_title = isbn_to_title[similar_isbn]
        distance = round(distances.flatten()[i], 2)
        similar_books.append([similar_title, distance])

    # Sort the similar books by distance to ensure consistency in output order
    similar_books = sorted(similar_books, key=lambda x: x[1], reverse=True)

    # Ensure to return the top 5 recommendations
    recommended_books.append(similar_books[:5])

    return recommended_books

In [22]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['I Know This Much Is True', 0.77], ['The Surgeon', 0.77], ['The Weight of Water', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
